In [12]:
#!pip install pyarrow

# import libraries
import numpy as np
import pandas as pd
from lightgbm.sklearn import LGBMRanker
import pickle


# import preprocessed dataset
train = pd.read_parquet('./train_deploy.parquet')


def get_session_lengths(df):
    return df.groupby('session')['session'].count().rename('session_length').to_numpy()

session_lengths_train = get_session_lengths(train)

feature_cols = train.columns[2:-2]
target = 'ground_truth'

X_train = train[feature_cols]
y_train = train[target]


# instantiate model
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=20,
    importance_type='gain',
    random_state=42
)



# fit to train
ranker = ranker.fit(
    X_train,
    y_train,
    group=session_lengths_train,
)


# saving model using pickle
pickle.dump(ranker, open('model.pkl','wb'))